In [ ]:
import Pkg; Pkg.activate("."); #Pkg.instantiate()
Pkg.develop(["FUSE", "IMAS", "FRESCO", "QED", "VacuumFields"])
Pkg.add("ProgressMeter")
Pkg.update()
using MAT, EFIT, FRESCO, VacuumFields, QED, Plots
using FUSE, IMAS, CoordinateConventions
using LinearAlgebra, ProgressMeter

In [ ]:
NA = [CartesianIndex()]
# This creates a QED Build struct based on a FRESCO canvas.coils
# The voltage waveforms are set to be constant values that match V=IR
function Build(canvas, dd)

    coils = canvas.coils
    Mcc = [VacuumFields.mutual(c1, c2) for c1 in coils, c2 in coils]
    Ic = [VacuumFields.current_per_turn(c) for c in coils] #current per turn
    Rc = [VacuumFields.resistance(c) for c in coils];
    Vc = Ic .* Rc
    V_waveforms = [QED.Waveform(Vc[k]) for k in eachindex(Vc)]

    eqt = dd.equilibrium.time_slice[]
    eqt1d = eqt.profiles_1d
    Ip = eqt.global_quantities.ip

    # COIL MUTUALS
    # in COCOS 11, psi = -L * I
    Mpc = [-FRESCO.plasma_flux_at_coil(k, canvas) / Ip for k in eachindex(coils)]
    dMpc_dt = zero(Mpc) # How Mpc changes in time (like shape)... to test later

    # INTERNAL INDUCTANCE
    # compute from eqt1d, since we're using P' and FF', so it's unclear if cp1d is correct
    It = IMAS.cumtrapz(eqt1d.area, eqt1d.j_tor)
    Wp = 0.5 * IMAS.trapz(eqt1d.psi, It)
    Li = (2 * Wp / Ip^2)

    # EXTERNAL INDUCTANCE
    # in COCOS 11, psi = -L * I
    ψb = eqt.profiles_1d.psi[end]
    ψc = -sum(Mpc[k] * Ic[k] for k in eachindex(coils))
    Le = -(ψb - ψc) / Ip
    Lp = (Li + Le)
    @show Li, Le
    Pohm = dd.core_sources.source[:ohmic].profiles_1d[].electrons.power_inside[end]
    Ini = dd.core_profiles.global_quantities.current_non_inductive[]
    Iohm = Ip - Ini
    Rp = abs(Pohm / (Ip * Iohm))

    Vni = Rp * Ini

    return QED.QED_build(Ic, Vc, Rc, Mcc, Vni, Rp, Lp, Mpc, dMpc_dt, V_waveforms);
end

function update_build!(build, canvas, dd)
    coils = canvas.coils

    eqt = dd.equilibrium.time_slice[]
    eqt1d = eqt.profiles_1d
    Ip = eqt.global_quantities.ip

    for (k, coil) in enumerate(coils)
        build.Ic[k] =  VacuumFields.current_per_turn(coil)
        # in COCOS 11, psi = -L * I
        build.Mpc[k] = -FRESCO.plasma_flux_at_coil(k, canvas) / Ip
    end

    # INTERNAL INDUCTANCE
    # compute from eqt1d, since we're using P' and FF', so it's unclear if cp1d is correct
    It = IMAS.cumtrapz(eqt1d.area, eqt1d.j_tor)
    Wp = 0.5 * IMAS.trapz(eqt1d.psi, It)
    Li = (2 * Wp / Ip^2)

    # EXTERNAL INDUCTANCE
    # in COCOS 11, psi = -L * I
    ψb = eqt1d.psi[end]
    ψc = -sum(build.Mpc[k] * build.Ic[k] for k in eachindex(coils))
    Le = -(ψb - ψc) / Ip    
    build.Lp = (Li + Le)

    Pohm = dd.core_sources.source[:ohmic].profiles_1d[].electrons.power_inside[end]
    Ini = dd.core_profiles.global_quantities.current_non_inductive[]
    Iohm = Ip - Ini
    build.Rp = abs(Pohm / (Ip * Iohm))

    build.Vni = build.Rp * Ini

    return build
end

function update_from_fresco!(dd, canvas, profile)
    eq = dd.equilibrium
    eqt = eq.time_slice[]
    eqt1d = eqt.profiles_1d
    eq2d = resize!(eqt.profiles_2d, 1)[1]

    eqt.global_quantities.magnetic_axis.r = canvas.Raxis
    eqt.global_quantities.magnetic_axis.z = canvas.Zaxis
    eqt.global_quantities.psi_boundary = canvas.Ψbnd
    eqt.global_quantities.psi_axis = canvas.Ψaxis

    Npsi = length(eqt1d.psi)
    eqt1d.psi = range(canvas.Ψaxis, canvas.Ψbnd, Npsi)
    # p' doesn't change
    eqt1d.f_df_dpsi = profile.ffprime.(eqt1d.psi_norm) .* profile.ffp_scale

    fend = eq.vacuum_toroidal_field.b0[end] * eq.vacuum_toroidal_field.r0
    f2 = 2 * IMAS.cumtrapz(eqt1d.psi, eqt1d.f_df_dpsi)
    f2 .= f2 .- f2[end] .+ fend ^ 2
    eqt1d.f = sign(fend) .* sqrt.(f2)

    pend = eqt1d.pressure[end]
    eqt1d.pressure = IMAS.cumtrapz(eqt1d.psi, eqt1d.dpressure_dpsi)
    eqt1d.pressure .+= pend .- eqt1d.pressure[end]

    eq2d.grid_type.index = 1
    eq2d.grid.dim1 = collect(range(canvas.Rs[1], canvas.Rs[end], Npsi))
    eq2d.grid.dim2 = collect(range(canvas.Zs[1], canvas.Zs[end], Npsi))
    FRESCO.update_interpolation!(canvas)
    eq2d.psi = [canvas._Ψitp(r, z) for r in eq2d.grid.dim1, z in eq2d.grid.dim2]

    fw = IMAS.first_wall(dd.wall)
    IMAS.flux_surfaces(eqt, fw.r, fw.z)

    # Here we reset the equilibrium grids, then copy over some info to core_profiles
    # BCL 4/17/25: I think only the final line (ip) will get used in the current setup,
    #              but this makes sure it's consistent
    FUSE.latest_equilibrium_grids!(dd)
    cp1d = dd.core_profiles.profiles_1d[]
    rhoc = cp1d.grid.rho_tor_norm
    rhoe = eqt.profiles_1d.rho_tor_norm
    Jpar = IMAS.Jtor_2_Jpar(rhoe, eqt1d.j_tor, true, eqt)
    cp1d.j_total = IMAS.interp1d(rhoe, Jpar).(rhoc)
    cp1d.pressure = IMAS.interp1d(rhoe, eqt1d.pressure).(rhoc)
    dd.core_profiles.global_quantities.ip = [eqt.global_quantities.ip]

    return dd
end

function evolve!(dd, canvas, profile, build, QI, η0, Δt; do_plot=false, debug=false)
    eqt = dd.equilibrium.time_slice[]
    A0 = eqt.global_quantities.area
    pp0 = profile.pprime

    canvas.Ip = QED.Ip(QI)
    for (i, coil) in enumerate(canvas.coils)
        VacuumFields.set_current_per_turn!(coil, build.Ic[i])
    end

    FRESCO.set_flux_at_coils!(canvas)

    success = true
    try
        status = FRESCO.solve!(canvas, profile, 400, 3;
                               relax=0.1, tolerance=1e-7, debug,
                               control=:eddy, initialize_current=false)

        update_from_fresco!(dd, canvas, profile)

        if eqt.global_quantities.area / A0 < 0.1
            # plasma is too small. exit
            success = false
        end
    catch e
        # Something happened... often plasma got too small
        e isa InterruptException && rethrow(e)
        display(e)
        success = false
    end
    
    # Update QED structs
    QI = FUSE.qed_init_from_imas(dd; uniform_rho = 101, j_tor_from=:core_profiles, ip_from=canvas.Ip)
    update_build!(build, canvas, dd)
    τp = abs(build.Lp/build.Rp)
    Nt = ceil(Int, Δt/(1e-6*τp))
    # Evolve coupled system
    QI = QED.evolve(QI, η0, build, Δt, Nt)

    do_plot && display(plot(canvas))
    return dd, canvas, profile, build, QI
end

In [ ]:
#=================== LOAD INITIAL EQUILIBRIUM FROM GSEVOLVE INIT ======================#

T = transform_cocos(7,11)
init = matread("gsevolve_init.mat")["init"]

dd0 = IMAS.dd()
resize!(dd0.equilibrium.time_slice)
dd0.equilibrium.time=[0.0]
dd0.equilibrium.time_slice[].global_quantities.ip = T["I"]*init["cpasma"]
dd0.equilibrium.time_slice[].global_quantities.vacuum_toroidal_field.b0 = T["B"]*init["bzero"]
dd0.equilibrium.vacuum_toroidal_field.b0 = [T["B"]*init["bzero"]]
dd0.equilibrium.time_slice[].global_quantities.vacuum_toroidal_field.r0 = init["rzero"]
dd0.equilibrium.vacuum_toroidal_field.r0 = init["rzero"]
dd0.equilibrium.time_slice[].boundary.outline.r = init["rbbbs"][:]
dd0.equilibrium.time_slice[].boundary.outline.z = init["zbbbs"][:]
resize!(dd0.equilibrium.time_slice[].boundary.x_point,1)
dd0.equilibrium.time_slice[].boundary.x_point[1].r = init["rbbbs"][1]
dd0.equilibrium.time_slice[].boundary.x_point[1].z = init["zbbbs"][1]
psi_norm = init["psibar"][:]
psibry = init["psibry"]
psimag = init["psimag"]
dd0.equilibrium.time_slice[].global_quantities.psi_axis = T["PSI"]*psimag
dd0.equilibrium.time_slice[].global_quantities.psi_boundary = T["PSI"]*psibry
psi = psimag .+ (psibry - psimag)*psi_norm
dd0.equilibrium.time_slice[].profiles_1d.psi = T["PSI"]*psi
dd0.equilibrium.time_slice[].profiles_1d.psi_norm = psi_norm
dd0.equilibrium.time_slice[].profiles_1d.rho_tor_norm = init["rhot"][:]
dd0.equilibrium.time_slice[].profiles_1d.f_df_dpsi = T["F_FPRIME"]*init["ffprim"][:]
dd0.equilibrium.time_slice[].profiles_1d.f = T["F"]*init["fpol"][:]
dd0.equilibrium.time_slice[].profiles_1d.dpressure_dpsi = T["PPRIME"]*init["pprime"][:]
dd0.equilibrium.time_slice[].profiles_1d.pressure = T["P"]*init["pres"][:]

eq2d = resize!(dd0.equilibrium.time_slice[].profiles_2d,1)[1]
eq2d.grid_type.index = 1
eq2d.grid.dim1 = init["rg"][:]
eq2d.grid.dim2 = init["zg"][:]
eq2d.psi = T["PSI"]*permutedims(init["psizr"])
eq2d.j_tor = 1e6*T["I"]*permutedims(init["jphi"]);


In [ ]:
function combine_coils!(dd::IMAS.dd, ks::Tuple{Int,Int}, orientation::Tuple{Int,Int}; clean::Union{Nothing, Symbol}=nothing)
    @assert clean in (nothing, :element, :coil)
    coils = dd.pf_active.coil
    k1, k2 = ks
    o1, o2 = orientation
    N1, N2 = length(coils[k1].element), length(coils[k2].element)
    append!(coils[k1].element, coils[k2].element)
    for i in 1:N1
        coils[k1].element[i].turns_with_sign = o1 * abs(coils[k1].element[i].turns_with_sign)
    end
    for i in (N1+1):(N1+N2)
        coils[k1].element[i].turns_with_sign = o2 * abs(coils[k1].element[i].turns_with_sign)
    end
    if clean === :element
        empty!(coils[k2].element)
    elseif clean === :coil
        deleteat!(coils, k2)
    end
end

In [ ]:
#=================== LOAD ODS ITER AND MERGE WITH GSEVOLVE DD ======================#

ini, act = FUSE.case_parameters(:ITER, init_from=:ods);
act.ActorFRESCO.nR = 33
act.ActorFRESCO.nZ = 65

dd = IMAS.dd();
FUSE.init(dd, ini, act);

empty!(dd.equilibrium)
merge!(dd,deepcopy(dd0))
fw = IMAS.first_wall(dd.wall)
IMAS.flux_surfaces(dd.equilibrium.time_slice[], fw.r, fw.z)

# Set initial coil currents
ic = init["ic"]
Nt = init["fcturn"]
for i=1:14
    VacuumFields.set_current_per_turn!(dd.pf_active.coil[i],ic[i])
end

@assert ic[3] == ic[4]
@assert ic[13] == -ic[14]

dd1 = dd

combine_coils!(dd1, (3, 4), (1, 1))
combine_coils!(dd1, (13, 14), (1, -1))
deleteat!(dd1.pf_active.coil, 14)
deleteat!(dd1.pf_active.coil, 4)

canvas = FRESCO.Canvas(dd1,33)
profile = FRESCO.PprimeFFprime(dd1);
eqt = dd1.equilibrium.time_slice[];
cp1d = dd1.core_profiles.profiles_1d[];
η0 = FUSE.η_imas(cp1d);
QI = FUSE.qed_init_from_imas(dd1; uniform_rho = 101);
FRESCO.solve!(canvas, profile, 200, 3; control = :shape, relax=0.1, tolerance=1e-6,
              debug=true, initialize_current=true, initialize_mutuals=true);
dd1 = update_from_fresco!(dd1,canvas,profile)
#p1 = plot(canvas)

#dI_12 = 1e5;
#VacuumFields.set_current_per_turn!(canvas.coils[12], dI_12)
#FRESCO.solve!(canvas, profile, 100, 3; relax=0.25, tolerance=1e-6, debug=true, control=:eddy, initialize_current=false, initialize_mutuals=true);
#p2 = plot(canvas)
#display(plot(p1,p2))

build = Build(canvas, dd1)

function time_evolution(dd, canvas, profile, build, QI, η0, Δt, Nt; debug=false)
    Ip = zeros(Nt)
    Ic = zeros(length(build.Ic), Nt)
    Z0 = zeros(Nt)
    Zcur = zeros(Nt)
    ps = Plots.Plot{Plots.GRBackend}[]
    tvmax = 10e-3
    V12 = t -> (t <= tvmax) ? -1e3 : 0.0
    @showprogress for i=1:Nt
        tstart = (i-1) * Δt
        for k in eachindex(build.V_waveforms)
            build.V_waveforms[k] = QED.Waveform{Float64}(t->0.0)
        end
        if tstart < tvmax
            build.V_waveforms[12] = QED.Waveform{Float64}(t -> V12(tstart + t))
        end
        dd, canvas, profile, build, QI = evolve!(dd, canvas, profile, build, QI, η0, Δt; do_plot=false, debug=(i==1000))
        Ip[i] = QED.Ip(QI)
        Ic[:, i] .= build.Ic
        Z0[i] = canvas.Zaxis
        Zcur[i] = sum(canvas._Jt .* canvas.Zs[NA,:]) / sum(canvas._Jt)
        push!(ps, plot(canvas, xrange=(1.0, 12.5), yrange=(-9,9), colorbar=false))
        if debug
            println("t = ",i*Δt)
            println("  Ip = ",Ip[i])
            println("  Z0 = ",Z0[i])
            println("  Vni = ",build.Vni)
            println("  Lp = ",build.Lp)
            println("  Rp = ",build.Rp)
            println("  Lp/Rp = ",build.Lp/build.Rp)
        end
    end
    return Ip, Ic, Z0, Zcur, ps
end

Δt = 0.001
Nt = 400
#Ip, Ic, Z0, Zcur, ps = time_evolution(dd1, canvas, profile, build, QI, η0, Δt, Nt);

In [ ]:
build.Mpc

In [ ]:
for (k,p) in enumerate(ps)
    plot!(p, title="t=$(round(1e3k*Δt)) ms", xlabel="", ylabel="", link=:both, xrange=(1,13),yrange=(-8,8))
end
k = length(ps)
plot!(ps[end], title="Final t=$(round(1e3k*Δt)) ms", xlabel="", ylabel="", link=:both, xrange=(1,13),yrange=(-8,8))

dN = Nt ÷ 10
if length(ps) == Nt
    display(plot(ps[dN:dN:end]..., layout=(2,5), size=(1300,700)))
else
    display(plot(vcat(ps[dN:dN:end], ps[end])..., layout=(2,5), size=(1300,700)))
end

In [ ]:
ts = range(Δt, Δt*Nt, Nt)
#=
#plot(ts[2:end], diff(log.(Z0[1] .- Z0)) ./ diff(ts))
plot(ts, log.(abs.(Z0[1] .- Z0)), lw=3)
#plot(1e3 .* ts, Z0[1] .- Z0)
gamma = 570e-3
plot!(ts, -3.5 .+ gamma .* ts, lw=2, ls=:dash)
gamma = 5000e-3
plot!(ts, -3.5 .+ gamma .* ts, lw=2, ls=:dash, legend=:none)
#display(plot!())
=#
#plot(100 .+ ts, Zcur, lw=3)
#pz = plot(100 .+ ts, Z0, lw=3)
#scatter!(pz, [100.05, 100.29], [0.536137, 0.66417])
plot!(pz, 100 .+ ts, Z0, lw=3)
plot!(pz,xrange=(100,100.4), yrange=(0.5,0.8))

In [ ]:
toksys = MAT.matread("vde.mat")
plot!(toksys["time"], toksys["zcur"], color=:black)


In [ ]:
plot(Ic[12,:])

In [ ]:
mean(u) = sum(u) / length(u)
function inst_growth_rate(t, u; window::Int = 21)
    @assert isodd(window) ≥ 1 "window must be an odd integer ≥ 3"
    n      = length(t)
    @assert n == length(u)  "t and u must have same length"

    half   = window ÷ 2
    logu   = log.(abs.(u))
    γ      = fill(NaN, n)

    for i in 1:n
        lo = max(1, i - half)
        hi = min(n, i + half)
        idx = lo:hi

        # subtract mean t to improve conditioning
        t_local = t[idx] .- mean(t[idx])
        A       = [t_local ones(length(idx))]
        coeffs  = A \ logu[idx]        # [γ, intercept]
        γ[i]    = coeffs[1]
    end
    return γ
end

plot(ts, inst_growth_rate(ts, Z0 .- Z0[1]; window=3), yrange=(0,20))
plot!(t_ts .- t_ts[1], inst_growth_rate(t_ts .- t_ts[1], z_ts .- z_ts[1]; window=11))

In [ ]:
ΔZ = Z0 .- Z0[1]
pg = plot(ts, log.(abs.(ΔZ)), lw=3)
gamma = 1.0 / 570e-3
plot!(pg, ts, -4.2 .+ gamma .* ts, lw=2, ls=:dash)
gamma = 3 * gamma
plot!(pg, ts, -4.2 .+ gamma .* ts, lw=2, ls=:dash, legend=:none)
@show(1.0 / gamma)

ΔZ = z_ts .- z_ts[1]
tts = t_ts .- t_ts[1]
plot!(tts, log.(abs.(ΔZ)), lw=3, color=:black)
plot!(pg, ts, -3.4 .+ gamma .* ts, lw=2, ls=:dash, color=:black, legend=:none)
@show(1.0 / gamma)

display(pg)

In [ ]:
t_ts

In [ ]:
ini, act = FUSE.case_parameters(:ITER, init_from=:ods);
act.ActorFRESCO.nR = 33
act.ActorFRESCO.nZ = 65

dd = IMAS.dd();
FUSE.init(dd, ini, act);

empty!(dd.equilibrium)
merge!(dd,deepcopy(dd0))
fw = IMAS.first_wall(dd.wall)
IMAS.flux_surfaces(dd.equilibrium.time_slice[], fw.r, fw.z)

# Set initial coil currents
ic = init["ic"]
Nt = init["fcturn"]
for i=1:14
    VacuumFields.set_current_per_turn!(dd.pf_active.coil[i],ic[i])
end

dd1 = dd

combine_coils!(dd1, (3, 4), (1, 1))
combine_coils!(dd1, (13, 14), (1, -1))
deleteat!(dd1.pf_active.coil, 14)
deleteat!(dd1.pf_active.coil, 4) 

canvas = FRESCO.Canvas(dd1,33,65)
profile = FRESCO.PprimeFFprime(dd1);
eqt = dd1.equilibrium.time_slice[];
cp1d = dd1.core_profiles.profiles_1d[];
η0 = FUSE.η_Jardin(cp1d,0);
QI = FUSE.qed_init_from_imas(dd1; uniform_rho = 101);

FRESCO.solve!(canvas, profile, 100, 3; control = :shape, relax=0.25, tolerance=1e-6,
              debug=false, initialize_current=true, initialize_mutuals=true);
dd1 = update_from_fresco!(dd1,canvas,profile)

build = Build(canvas, dd1)
R = diagm(build.Rc[13:end])
M = build.Mcc[13:end, 13:end]
A = inv(M) * R
τ = 1.0 ./ eigvals(A)
maximum(τ)